# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [19]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings.csv',low_memory=False)

In [21]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [22]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,runtime,budget,genres,original_language,original_title,revenue,id,popularity
0,81.0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,373554033.0,862,21.946943
1,104.0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,Jumanji,262797249.0,8844,17.015539
2,101.0,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,Grumpier Old Men,0.0,15602,11.7129
3,127.0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,81452156.0,31357,3.859495
4,106.0,0,"[{'id': 35, 'name': 'Comedy'}]",en,Father of the Bride Part II,76578911.0,11862,8.387519


In [23]:
#Tirando budgets diferente de 0 (budget ainda como string
df=df[df.budget != "0"]
df=df.drop([19730, 29503,35587])

In [24]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

runtime              float64
budget               float64
genres                object
original_language     object
original_title        object
revenue              float64
id                   float64
popularity            object
dtype: object

In [25]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df.id=df.id.astype('int')

In [26]:
#Tirando media dos ratings
imdb=imdb.sort_values(['movieId'], ascending=[True])
MediaRating=imdb.drop("userId",1,inplace=True)
MediaRating=imdb.drop("timestamp",1,inplace=True)
imdb.columns

Index(['movieId', 'rating'], dtype='object')

In [27]:
imdb.tail()

,movieId,rating
18234777,176267,4.0
9448563,176269,3.5
83813,176271,5.0
25735653,176273,1.0
17181377,176275,3.0


In [28]:
rating=pd.Series(imdb.groupby(['movieId'])['rating'].mean())

In [29]:
rating=rating.reset_index()
rating = rating.rename(columns={'movieId': 'id'})
rating=rating.set_index('id')

In [30]:
df=df.set_index("id")

In [34]:
dados_filtrados=pd.merge(df, rating, on='id', how='inner')
dados_filtrados

,runtime,budget,genres,original_language,original_title,revenue,popularity,rating
id,,,,,,,,
5,98.0,4000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",en,Four Rooms,4.300000e+06,9.026586,3.079565
11,121.0,11000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,Star Wars,7.753980e+08,42.149697,3.660591
12,100.0,94000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",en,Finding Nemo,9.403355e+08,25.497794,2.672179
13,142.0,55000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Forrest Gump,6.779454e+08,48.307194,3.326442
14,122.0,15000000.0,"[{'id': 18, 'name': 'Drama'}]",en,American Beauty,3.562966e+08,20.726578,3.430807
15,119.0,839727.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",en,Citizen Kane,2.321767e+07,15.811921,2.727040
16,140.0,12800000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",en,Dancer in the Dark,4.003188e+07,10.684806,3.795511
18,126.0,90000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,The Fifth Element,2.639202e+08,24.30526,3.409031
19,153.0,92620000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",de,Metropolis,6.504220e+05,14.487867,2.633983


In [38]:
dados_filtrados.to_csv("dataframe_filtrado.csv")